## 7. Método de Kane

In [1]:
from sympy import symbols, sin, cos
from sympy.physics.mechanics import ReferenceFrame, inertia, Point, dynamicsymbols
from sympy.physics.mechanics import RigidBody, KanesMethod
import numpy as np

In [42]:
# Para el ejemplo del movimiento de un cilindro sobre una rampa
t,m,g,I,R,theta,=symbols('t,m,g,I,R,theta')
q1,q2=dynamicsymbols('q1,q2')

a=ReferenceFrame('A')
d=a.orientnew('D','Axis',(theta,a.z))
c=d.orientnew('C','Axis',(q2,d.z))

#Defina puntos de interes y velocidades
o=Point('O')
c_com=o.locatenew('C_com',d.x*q1)
o.set_vel(a,0)
c_com.set_vel(c,0)

Ic=inertia(c,0,0,I)

#Defina un cuerpo rigido para el cilindro
body_cilindro=RigidBody('cilindro',c_com,c,m,(Ic,c_com))

u1,u2=dynamicsymbols('u1,u2')
km=KanesMethod(a,[q1,q2],[u1,u2],kd_eqs=[u1-q1.diff(t),u2-R*q2.diff(t)])

In [55]:
# Calcule las fuerzas inerciales generalizadas
(fr,frstar)=km.kanes_equations(bodies=[body_cilindro],loads=[(c_com,-m*g*a.y)])

Matrix([
[          -u1(t) + Derivative(q1(t), t)],
[         Derivative(q2(t), t) - u2(t)/R],
[g*m*sin(theta) + m*Derivative(u1(t), t)],
[            I*Derivative(u2(t), t)/R**2]])

In [57]:
frstar

Matrix([
[     -m*Derivative(u1(t), t)],
[-I*Derivative(u2(t), t)/R**2]])

In [58]:
# Observe las equaciones de movimiento 
km.mass_matrix_full*km.q.col_join(km.u).diff(t)-km.forcing_full

Matrix([
[          -u1(t) + Derivative(q1(t), t)],
[         Derivative(q2(t), t) - u2(t)/R],
[g*m*sin(theta) + m*Derivative(u1(t), t)],
[            I*Derivative(u2(t), t)/R**2]])

In [71]:
# Para el ejemplo del modelo de un tren de aterrizaje

# Defina las variables de movimiento
# y parametros del modelo
q1,q2=dynamicsymbols('q1,q2')
# Parametros del modelo
h,d,mb,mc,Ia,Ir,g,M3=symbols('h,d,mb,mc,Ia,Ir,g,M3')

#Marcos de referencia
a=ReferenceFrame('A')
b=a.orientnew('B','Axis',(q1,a.z))
c=b.orientnew('C','Axis',(q2,b.x))

# Puntos de interes
o=Point('O')
b_com=o.locatenew('Bcom',-h*b.y/2)
b_end=o.locatenew('Bend',-h*b.y)
c_com=b_end.locatenew('Ccom',d*b.x)

# Defina la velocidad de cada punto
o.set_vel(a,0)
b_com.set_vel(b,0)
c_com.set_vel(c,0)
b_com.v2pt_theory(o,a,b)
c_com.v2pt_theory(b_end,b,c)

#Propiedades inerciales
ic=inertia(c, Ia, Ir, Ir)

# Cuerpos rigidos
body_llanta=RigidBody('llanta',c_com,c,mc,(ic,c_com))

u1,u2=dynamicsymbols('u1,u2')
km=KanesMethod(a,[q1,q2],[u1,u2],kd_eqs=[u1-q1.diff(t),u2-q2.diff(t)])
(fr,frstar)=km.kanes_equations(bodies=[body_llanta],loads=[(b,M3*b.z)])

# Observe las equaciones de movimiento 
eom=km.mass_matrix_full*km.q.col_join(km.u).diff(t)-km.forcing_full
eom.simplify()
eom

Matrix([
[                      -u1(t) + Derivative(q1(t), t)],
[                      -u2(t) + Derivative(q2(t), t)],
[-M3 + (Ir + d**2*mc + h**2*mc)*Derivative(u1(t), t)],
[                            Ia*Derivative(u2(t), t)]])